In [1]:
import pdfplumber as pdfwork
import pystardict
from bs4 import BeautifulSoup
# import json

In [2]:
path = '/Users/anubis/Desktop/Personal_Projects/Learning Spanish/Spanish Notes.pdf'
text = ''
with pdfwork.open(path) as pdf:
	for page in pdf.pages:
		text += page.extract_text() + '\n'
words = text.splitlines()
words = words[2:]
print(words)

['Agradecer = to thank', 'Apoyar = support', 'Chusma = mob', 'Chido = cool', 'Cobrado = charged', 'Cartulinas = cardboard', 'Pliego = sheet', 'Chafa = knockoff', 'Abarrotes = grcoeries', 'Ventaja = advantage', 'Chingo de cosas = fuck ton of things', 'Negocio = business', 'SIGUE SIENDO = REMAINS', 'Cuerda floja = hanging on by a thread', 'Orgullo = pride', 'Pinche = lousy but colloquial', 'Duda = doubt', 'Cadena = chain', 'Tal vez = maybe', 'TRAS = AFTER', 'Ep 189 - La Basura', 'Wey = Dude', 'Barrenderos = street sweepers', 'Escobas = brooms', 'La neta = the truth', 'cantidad = quantity', 'porqueria = crap', 'puse = i put, conjugation of poner in the past participio', 'suena = sounds', 'relleno sanitario = landfill', 'pepenadores = scavengers', 'quemas = burn', 'abastos = supplies', 'basurero = garbage collector', 'un sueldo = salary', 'desmadre = chaos', 'lata = can', 'inundar = flood', 'campana = bell', 'agarrar = to grab', 'tiliche = junk', 'colchon = mattress', 'engaña = to trick or

In [3]:
tuples = []
for phrase in words:
	pieces = phrase.split(' = ')
	if len(pieces) == 2:
		tuples.append((pieces[0].lower(), pieces[1]))
jsons = []
for tup in tuples:
	jsons.append({"phrase": tup[0], "vedDesc": tup[1], "definition": None, "similarWords": None, "partOfSpeech": None})
jsons

[{'phrase': 'agradecer',
  'vedDesc': 'to thank',
  'definition': None,
  'similarWords': None,
  'partOfSpeech': None},
 {'phrase': 'apoyar',
  'vedDesc': 'support',
  'definition': None,
  'similarWords': None,
  'partOfSpeech': None},
 {'phrase': 'chusma',
  'vedDesc': 'mob',
  'definition': None,
  'similarWords': None,
  'partOfSpeech': None},
 {'phrase': 'chido',
  'vedDesc': 'cool',
  'definition': None,
  'similarWords': None,
  'partOfSpeech': None},
 {'phrase': 'cobrado',
  'vedDesc': 'charged',
  'definition': None,
  'similarWords': None,
  'partOfSpeech': None},
 {'phrase': 'cartulinas',
  'vedDesc': 'cardboard',
  'definition': None,
  'similarWords': None,
  'partOfSpeech': None},
 {'phrase': 'pliego',
  'vedDesc': 'sheet',
  'definition': None,
  'similarWords': None,
  'partOfSpeech': None},
 {'phrase': 'chafa',
  'vedDesc': 'knockoff',
  'definition': None,
  'similarWords': None,
  'partOfSpeech': None},
 {'phrase': 'abarrotes',
  'vedDesc': 'grcoeries',
  'definitio

In [4]:
path = '/Users/anubis/Desktop/Personal_Projects/Learning Spanish/EnEsDict/es-en.enwikt'
dict = pystardict.Dictionary(path)
example = 'estar'
if example in dict:
	definition = dict[example]
soup = BeautifulSoup(definition)
soup

<html><body><b>estar</b> <i>verb</i>
<ol style="padding:0; margin-left: 1em; margin-top: .2em; margin-bottom: 1em">
<li>[<i>intransitive</i>] to be (have a temporary or permanent location in space)</li>
<li>[<i>intransitive</i>] to be (denotes a copula, in a transient fashion)</li>
<li>to be (auxiliary verb for the progressive/continuous aspect, preceding the gerund of the verb)</li>
<li>[<i>intransitive</i>] to be in a state (in a passive voice sense)</li>
<li>[<i>reflexive, followed by adjective</i>] to be, stay (denotes a copula, in a transient fashion)</li>
<li>[<i>intransitive</i>] to be (still) {{+obj|es|:por(inf)&lt;to be done&gt;}}</li>
<li>[<i>intransitive</i>] to be in a long-term state (in specific idioms)</li>
</ol>
<p style="margin-top: 1em"><i>Note:</i> See ser. Contrast also with quedar.</p><p style="margin-top: 1em"><i>Etymology:</i> Inherited from Old Spanish "estar", inherited from Latin "stāre" (“stand”), from Proto-Indo-European "*steh₂-" (compare English "stand"). 